In [ ]:
# imports
import requests
import pandas as pd


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
def get_foursquare_data(latitude, longitude, foursquare_api_key, radius=1000, limit=50):
    url = 'https://api.foursquare.com/v3/places/search'
    headers = {
        'Accept': 'application/json',
        'Authorization': foursquare_api_key
    }
    params = {
        'll': f'{latitude},{longitude}',
        'radius': radius,
        'limit': limit
    }
    response = requests.get(url, params=params, headers=headers)
    if response.status_code != 200:
        raise Exception('Error with Foursquare API call')
    return response.json()


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating